# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import pickle

# Kriging 3D Interpolation

### Define Functions

In [ ]:
def plot_3D_mesh(t, grid_x, grid_y, grid_z, title: str, color_scale='Viridis'):
    GZ, GY, GX = np.meshgrid(grid_z, grid_y, grid_x, indexing='ij')

    fig = go.Figure(
        data=[go.Volume(
            x=GX.flatten(),
            y=GY.flatten(),
            z=GZ.flatten(),
            value=t.flatten(),
            isomax=t.max(),
            isomin=t.min(),
            opacity=0.5,  
            surface_count=25,  # Adjust the number of isosurfaces
            colorscale=color_scale 
        )])

    # Update the layout of the plot
    fig.update_layout(
        height=600,
        width=700,
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis=dict(
                title='Depth',
                autorange='reversed'  # Automatically reverse the z-axis
            )
        )
    )

    # Show the plot
    fig.show()

In [ ]:
def load_data(name: str) -> pd.DataFrame:
    with open(f'data/t_{name}.pkl', 'rb') as f:
        interpolated_data = pickle.load(f)
    with open(f'data/ss3d_{name}.pkl', 'rb') as f:
        varience_data = pickle.load(f)
    return interpolated_data, varience_data

In [ ]:
def load_grid() -> np.ndarray:
    with open(f'data/grid.pkl', 'rb') as f:
        grid = pickle.load(f)
    return grid

### Load Data

In [ ]:
grid = load_grid()
t_poro, ss3d_poro = load_data('poro')
t_hydr, ss3d_hydr = load_data('hydr')

### Kriging Interpolation of Porosity

In [ ]:
plot_3D_mesh(t_poro, *grid, "Kriging Interpolation of Porosity", "Plasma")

In [ ]:
plot_3D_mesh(ss3d_poro, *grid, "Kriging Varience of Porosity")

### Kriging Interpolation of Hydrate Saturation

In [ ]:
plot_3D_mesh(t_hydr, *grid, "Kriging Interpolation of Hydrate Saturation", "Plasma")

In [ ]:
plot_3D_mesh(ss3d_hydr, *grid, "Kriging Varience of Hydrate Saturation")

# Resource Distribution

In [ ]:
def estimate_resource(sensor_data: pd.Series) -> float:
    """Estimate the resource at a given location based on sensor data"""
    # Get the Porosity and the Hydrate saturation
    porosity = sensor_data['Porosity']
    hydrate_saturation = sensor_data['Hydrate Saturation']

    valid_volume = 1 # Assume the valid volume is 1 cubic meter
    factor = 155 # Assume the factor is 155

    # Calculate the resource estimate
    return valid_volume * porosity * hydrate_saturation * factor

In [ ]:
# Calculate the resource estimate for each point in the grid
resource_estimate = np.zeros_like(t_poro)
for i in range(t_poro.shape[0]):
    for j in range(t_poro.shape[1]):
        for k in range(t_poro.shape[2]):
            sensor_data = pd.Series({
                'Porosity': t_poro[i, j, k],
                'Hydrate Saturation': t_hydr[i, j, k]
            })
            resource_estimate[i, j, k] = estimate_resource(sensor_data)

In [ ]:
# Plot the resource estimate
plot_3D_mesh(resource_estimate, *grid, "Resource Estimate", "Plasma")

# Total Resources

In [ ]:
from loadAndPreprocess import load_and_preprocess
well_info, sensor_data_list = load_and_preprocess()

# Remove the data point with NaN value
well_info = well_info.dropna()
for sensor_data in sensor_data_list:
    sensor_data.dropna(inplace=True)
    
X = well_info['X'].values
Y = well_info['Y'].values
Z = [sensor_data['Depth'] for sensor_data in sensor_data_list]
Z = np.concatenate(Z)

l = (max(X) - min(X)) / len(grid[0])
w = (max(Y) - min(Y)) / len(grid[1])
h = (max(Z) - min(Z)) / len(grid[2])

In [ ]:
# Calculate the sum of the total resource estimate
total_resource_estimate = resource_estimate.sum() * l * w * h
print(f"Total Resource Estimate: {total_resource_estimate:.2f} cubic meters")